# Ag3 cloud data access

This page provides information about how to access data from [*Anopheles gambiae* 1000 Genomes project (Ag1000G) phase 3](intro) via Google Cloud. This includes sample metadata and single nucleotide polymorphism (SNP) calls.

This notebook illustrates how to read data directly from the cloud, without having to first download any data locally. This notebook can be run from any computer, but will work best when run from a compute node within Google Cloud, because it will be physically closer to the data and so data transfer is faster. For example, this notebook can be run via [Google Colab](https://colab.research.google.com/) which are free interactive computing service running in the cloud.

To launch this notebook in the cloud and run it for yourself, click the launch icon (<i class="fas fa-rocket"></i>) at the top of the page and select one of the cloud computing services available.

## Data hosting

All data required for this notebook is hosted on Google Cloud Storage (GCS). Data are hosted in the `vo_agam_release` bucket, which is a multi-region bucket located in the United States. All data hosted in GCS are publicly accessible and do not require any authentication to access. 

## Setup

Running this notebook requires some Python packages to be installed:

In [ ]:
!pip install -q malariagen_data

To make accessing these data more convenient, we've created the [malariagen_data](https://github.com/malariagen/malariagen-data-python) Python package. This is experimental so please let us know if you find any bugs or have any suggestions. See the [Ag3 API docs](api) for documentation of all functions available from this package. 

Import the packages we'll need to use here.

In [2]:
import numpy as np
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})
import allel
import malariagen_data

Ag3 data access from Google Cloud is set up with the following code:

In [3]:
ag3 = malariagen_data.Ag3("gs://vo_agam_release/")

## Sample sets

Data in this release are organised into 28 sample sets. Each of these sample sets corresponds to a set of mosquito specimens contributed by a collaborating study. Depending on your objectives, you may want to access data from only specific sample sets, or all sample sets.

To see which sample sets are available, load the sample set manifest into a pandas dataframe:

In [4]:
df_sample_sets = ag3.sample_sets()
df_sample_sets

,sample_set,sample_count,release
0,AG1000G-AO,81,v3
1,AG1000G-BF-A,181,v3
2,AG1000G-BF-B,102,v3
3,AG1000G-BF-C,13,v3
4,AG1000G-CD,76,v3
5,AG1000G-CF,73,v3
6,AG1000G-CI,80,v3
7,AG1000G-CM-A,303,v3
8,AG1000G-CM-B,97,v3
9,AG1000G-CM-C,44,v3


For more information about these sample sets, see the section on sample sets in the [introduction to Ag1000G phase 3](intro).

## Sample metadata

Data about the samples that were sequenced to generate this data resource are available, including the time and place of collection, the gender of the specimen, and our call regarding the species of the specimen. These are organised by sample set.

E.g., load sample metadata for the AG1000G-BF-A and AG1000G-BF-B sample sets into a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe):

In [5]:
df_samples = ag3.sample_metadata(sample_sets=["AG1000G-BF-A", "AG1000G-BF-B"])
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,release,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii,species
0,AB0085-Cx,BF2-4,Austin Burt,Burkina Faso,Pala,2012,7,11.150,-4.235,F,AG1000G-BF-A,v3,0.024,0.002,gamb_colu,gambiae,gambiae
1,AB0086-Cx,BF2-6,Austin Burt,Burkina Faso,Pala,2012,7,11.150,-4.235,F,AG1000G-BF-A,v3,0.038,0.002,gamb_colu,gambiae,gambiae
2,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana,2012,7,11.233,-4.472,F,AG1000G-BF-A,v3,0.982,0.002,gamb_colu,coluzzii,coluzzii
3,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana,2012,7,11.233,-4.472,F,AG1000G-BF-A,v3,0.990,0.002,gamb_colu,coluzzii,coluzzii
4,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana,2012,7,11.233,-4.472,F,AG1000G-BF-A,v3,0.975,0.002,gamb_colu,coluzzii,coluzzii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,AB0533-C,BF13-18,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.235,-4.535,F,AG1000G-BF-B,v3,0.021,0.002,gamb_colu,gambiae,gambiae
279,AB0536-C,BF13-31,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.235,-4.535,F,AG1000G-BF-B,v3,0.025,0.002,gamb_colu,gambiae,gambiae
280,AB0537-C,BF13-32,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.235,-4.535,F,AG1000G-BF-B,v3,0.029,0.002,gamb_colu,gambiae,gambiae
281,AB0538-C,BF13-33,Austin Burt,Burkina Faso,Souroukoudinga,2014,7,11.235,-4.535,F,AG1000G-BF-B,v3,0.018,0.002,gamb_colu,gambiae,gambiae


The `sample_id` column gives the sample identifier used throughout all Ag1000G analyses.

The `country`, `location`, `latitude` and `longitude` columns give the location where the specimen was collected.

The `year` and `month` columns give the approximate date when the specimen was collected.

The `sex_call` column gives the gender as determined from the sequence data.

All samples were derived from wild-caught mosquito specimens, with the exception of samples in AG1000G-X which were derived from lab crosses. For most analyses you may want to exclude the crosses and work with wild-caught samples only. To load metadata for all wild-caught sample sets (i.e., excluding AG1000G-X), you can use the shortcut "v3_wild", e.g.:

In [6]:
df_samples = ag3.sample_metadata(sample_sets="v3_wild")
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,release,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii,species
0,AR0047-C,LUA047,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.945,0.001,gamb_colu,coluzzii,coluzzii
1,AR0049-C,LUA049,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.933,0.001,gamb_colu,coluzzii,coluzzii
2,AR0051-C,LUA051,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.937,0.002,gamb_colu,coluzzii,coluzzii
3,AR0061-C,LUA061,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.938,0.002,gamb_colu,coluzzii,coluzzii
4,AR0078-C,LUA078,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.926,0.001,gamb_colu,coluzzii,coluzzii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,AC0295-C,K92,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.026,0.002,gamb_colu,gambiae,gambiae
2780,AC0296-C,K93,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.029,0.003,gamb_colu,gambiae,gambiae
2781,AC0297-C,K94,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.026,0.002,gamb_colu,gambiae,gambiae
2782,AC0298-C,K95,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.029,0.002,gamb_colu,gambiae,gambiae


[Pandas](https://pandas.pydata.org/) can be used to explore and query the sample metadata in various ways. E.g., here is a summary of the numbers of samples by species:

In [7]:
df_samples.groupby("species").size()

species
arabiensis                          368
coluzzii                            675
gambiae                            1571
intermediate_arabiensis_gambiae       1
intermediate_gambiae_coluzzii       169
dtype: int64

Note that samples within a sample set may belong to different species. For convenience, we have made a species call for all samples in Ag1000G phase 3, using the genomic data. Calling species is not always straightforward, and we have used two different methods for species calling, ancestry informative markers (AIM) and principal components analysis (PCA). When loading the sample metadata, the AIM species calls are included by default. The results of these two different methods generally agree, although there are some populations where results are different, particularly in Guinea-Bissau, The Gambia, Kenya and Tanzania. If you have any questions about how to interpret these species calls, please get in touch.

## SNP calls

Data on SNP calls, including the SNP positions, alleles, site filters, and genotypes, can be accessed as an [xarray Dataset](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset).

E.g., access SNP calls for chromosome arm 3R for all wild-caught samples.

In [8]:
ds_snps = ag3.snp_calls(contig="3R", sample_sets="v3_wild")
ds_snps

,Array,Chunk
Bytes,208.91 MB,2.10 MB
Shape,"(52226568,)","(524288,)"
Count,101 Tasks,100 Chunks
Type,int32,numpy.ndarray
,Array,Chunk
Bytes,52.23 MB,524.29 kB
Shape,"(52226568,)","(524288,)"
Count,100 Tasks,100 Chunks
Type,uint8,numpy.ndarray
,Array,Chunk


The arrays within this dataset are backed by [Dask arrays](https://docs.dask.org/en/latest/array.html), and can be accessed as shown below.

### SNP sites and alleles

We have called SNP genotypes in all samples at all positions in the genome where the reference allele is not "N". Data on this set of genomic positions and alleles for a given chromosome arm (e.g., 3R) can be accessed as [Dask arrays](https://docs.dask.org/en/latest/array.html) as follows.

In [9]:
pos = ds_snps["variant_position"].data
pos

,Array,Chunk
Bytes,208.91 MB,2.10 MB
Shape,"(52226568,)","(524288,)"
Count,101 Tasks,100 Chunks
Type,int32,numpy.ndarray


In [10]:
alleles = ds_snps["variant_allele"].data
alleles

,Array,Chunk
Bytes,208.91 MB,1.57 MB
Shape,"(52226568, 4)","(524288, 3)"
Count,502 Tasks,200 Chunks
Type,|S1,numpy.ndarray


Data can be loaded into memory as a [NumPy array](https://numpy.org/doc/stable/user/absolute_beginners.html) as shown in the following examples.

In [11]:
# read first 10 SNP positions into a numpy array
p = pos[:10].compute()
p

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)

In [12]:
# read first 10 SNP alleles into a numpy array
a = alleles[:10].compute()
a

array([[b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'T', b'A', b'C', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'T', b'A', b'C', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'T', b'A', b'C', b'G'],
       [b'T', b'A', b'C', b'G']], dtype='|S1')

Here the first column contains the reference alleles, and the remaining columns contain the alternate alleles.

Note that a byte string data type is used here for efficiency. E.g., the Python code `b'T'` represents a byte string containing the letter "T", which here stands for the nucleotide thymine.

Note that we have chosen to genotype all samples at all sites in the genome, assuming all possible SNP alleles. Not all of these alternate alleles will actually have been observed in the Ag3 samples. To determine which sites and alleles are segregating, an allele count can be performed over the samples you are interested in. See the example below. 

### Site filters

SNP calling is not always reliable, and we have created some site filters to allow excluding low quality SNPs. 

Because different species may have different genome accessibility issues, we have created three separate site filters:

* The "gamb_colu" filter is design for working only with *An. gambiae* and/or *An. coluzzii* samples. 
* The "arab" filter is designed for working with *An. arabiensis* samples. 
* The "gamb_colu_arab" filter is suitable for when analysing samples of any species together. 

Each set of site filters provides a "filter_pass" Boolean mask for each chromosome arm, where True indicates that the site passed the filter and is accessible to high quality SNP calling.

The site filters data can be accessed as dask arrays as shown in the examples below. 

In [13]:
# access gamb_colu_arab site filters for chromosome arm 3R as a dask array
filter_pass = ds_snps['variant_filter_pass_gamb_colu_arab'].data
filter_pass

,Array,Chunk
Bytes,52.23 MB,300.00 kB
Shape,"(52226568,)","(300000,)"
Count,176 Tasks,175 Chunks
Type,bool,numpy.ndarray


In [14]:
# read filter values for first 10 SNPs (True means the site passes filters)
f = filter_pass[:10].compute()
f

array([False, False, False, False, False, False, False, False, False,
       False])

### SNP genotypes

SNP genotypes for individual samples are available. Genotypes are stored as a three-dimensional array, where the first dimension corresponds to genomic positions, the second dimension is samples, and the third dimension is ploidy (2). Values coded as integers, where -1 represents a missing value, 0 represents the reference allele, and 1, 2, and 3 represent alternate alleles.

SNP genotypes can be accessed as dask arrays as shown below.

In [15]:
gt = ds_snps["call_genotype"].data
gt

,Array,Chunk
Bytes,290.80 GB,30.00 MB
Shape,"(52226568, 2784, 2)","(300000, 50, 2)"
Count,23827 Tasks,11900 Chunks
Type,int8,numpy.ndarray


Note that the columns of this array (second dimension) match the rows in the sample metadata, if the same sample sets were loaded. I.e.:

In [17]:
df_samples = ag3.sample_metadata(sample_sets="v3_wild")
gt = ds_snps["call_genotype"].data
len(df_samples) == gt.shape[1]

True

You can use this correspondance to apply further subsetting operations to the genotypes by querying the sample metadata. E.g.:

In [18]:
loc_gambiae = df_samples.eval("species == 'gambiae'").values
print(f"found {np.count_nonzero(loc_gambiae)} gambiae samples")
ds_snps_gambiae = ds_snps.isel(samples=loc_gambiae)
ds_snps_gambiae

found 1571 gambiae samples


,Array,Chunk
Bytes,208.91 MB,2.10 MB
Shape,"(52226568,)","(524288,)"
Count,101 Tasks,100 Chunks
Type,int32,numpy.ndarray
,Array,Chunk
Bytes,52.23 MB,524.29 kB
Shape,"(52226568,)","(524288,)"
Count,100 Tasks,100 Chunks
Type,uint8,numpy.ndarray
,Array,Chunk


Data can be read into memory as numpy arrays, e.g., read genotypes for the first 5 SNPs and the first 3 samples:

In [19]:
g = gt[:5, :3, :].compute()
g

array([[[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]]], dtype=int8)

If you want to work with the genotype calls, you may find it convenient to use [scikit-allel](http://scikit-allel.readthedocs.org/).
E.g., the code below sets up a genotype array.

In [20]:
# use the scikit-allel wrapper class for genotype calls
gt = allel.GenotypeDaskArray(ds_snps["call_genotype"].data)
gt

<GenotypeDaskArray shape=(52226568, 2784, 2) dtype=int8>

## Copy number variation (CNV) data

Data on copy number variation within the Ag3 cohort are available as three separate data types:

* **HMM** -- Genome-wide inferences of copy number state within each individual mosquito in 300 bp non-overlapping windows.
* **Coverage calls** -- Genome-wide copy number variant calls, derived from the HMM outputs by analysing contiguous regions of elevated copy number state then clustering of variants across individuals based on breakpoint proximity.
* **Discordant read calls** -- Copy number variant calls at selected insecticide resistance loci, based on analysis of read alignments at CNV breakpoints.

For more information on the methods used to generate these data, see the [variant-calling methods](methods) page.

The `malariagen_data` Python package provides some convenience functions for accessing these data, illustrated below.

### CNV HMM

Access HMM data via the `cnv_hmm()` method, which returns an [xarray](http://xarray.pydata.org/en/stable/) dataset. E.g.:

In [21]:
ds_hmm = ag3.cnv_hmm(contig="2R", sample_sets="AG1000G-TZ")
ds_hmm

<xarray.Dataset>
Dimensions:           (samples: 300, variants: 205151)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_end       (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(205151,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(300,), meta=np.ndarray>
Dimensions without coordinates: samples, variants
Data variables:
    call_CN           (variants, samples) int8 dask.array<chunksize=(65536, 64), meta=np.ndarray>
    call_RawCov       (variants, samples) int32 dask.array<chunksize=(65536, 64), meta=np.ndarray>
    call_NormCov      (variants, samples) float32 dask.array<chunksize=(65536, 64), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

Here "variants" are the 300 bp windows in which coverage was calculated and the HMM fitted. Window start positions are given by the `variant_position` array and ends are given by `variant_end`.

In [39]:
pos = ds_hmm['variant_position'].values
pos

array([       1,      301,      601, ..., 61544401, 61544701, 61545001],
      dtype=int32)

In [40]:
end = ds_hmm['variant_end'].values
end

array([     300,      600,      900, ..., 61544700, 61545000, 61545105],
      dtype=int32)

Copy number state is given by the `call_CN` array, where rows are windows and columns are individual samples.

On the autosomes (2R, 2L, 3R, 3L) normal diploid copy number is 2. Values greater than 2 mean amplification, less then 2 mean deletion.

On the X chromosome, normal copy number is 2 in females and 1 in males.

For all chromosomes, -1 means missing, i.e., the window was not included.

Rows are variants (windows), columns are individual samples.

In [41]:
cn = ds_hmm['call_CN'].values
cn

array([[ 5,  2,  2, ...,  6,  4,  2],
       [ 5,  2,  2, ...,  6,  4,  2],
       [ 5, 11,  8, ...,  6,  4,  2],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int8)

The `gene_cnv()` method can be used to summarise the HMM data and compute modal coverage by gene, for all genes in a chromosome arm. E.g.:

In [42]:
ds_gene_cnv = ag3.gene_cnv(contig="2R", sample_sets="AG1000G-TZ")
ds_gene_cnv

<xarray.Dataset>
Dimensions:        (genes: 3668, samples: 300)
Coordinates:
    gene_id        (genes) object 'AGAP001096' 'AGAP013094' ... 'AGAP004676'
    gene_start     (genes) int64 3426 13754 18496 ... 61303692 61372543 61478226
    gene_end       (genes) int64 9985 15149 19579 ... 61306248 61418536 61478802
    sample_id      (samples) object 'BL0046-C' 'BL0047-C' ... 'BL0250-CW'
Dimensions without coordinates: genes, samples
Data variables:
    gene_windows   (genes) int64 23 6 5 8 33 63 19 29 ... 168 546 57 80 10 154 3
    gene_name      (genes) object nan nan nan nan nan ... nan nan 'GPRMAC2' nan
    gene_strand    (genes) object '+' '+' '+' '+' '-' ... '+' '+' '-' '+' '+'
    CN_mode        (genes, samples) int8 2 2 2 2 2 2 2 2 2 ... 3 3 3 4 4 3 4 2 1
    CN_mode_count  (genes, samples) int64 23 23 23 23 23 23 23 ... 2 2 3 3 3 3 1

The `gene_cnv_frequencies()` method can be used to compute CNV frequencies by gene and cohort, e.g.:

In [43]:
cohorts = {
    'tz_gamb': "sample_set == 'AG1000G-TZ' and species == 'gambiae'",
    'tz_arab': "sample_set == 'AG1000G-TZ' and species == 'arabiensis'",
}
df_gene_cnv_freqs = ag3.gene_cnv_frequencies(contig='2R', cohorts=cohorts)
df_gene_cnv_freqs

,contig,start,end,strand,Name,description,tz_gamb_amp,tz_gamb_del,tz_arab_amp,tz_arab_del
ID,,,,,,,,,,
AGAP001096,2R,3426,9985,+,NaN,NaN,0.014706,0.058824,0.000000,0.004444
AGAP013094,2R,13754,15149,+,NaN,Elongation of very long chain fatty acids prot...,0.000000,0.044118,0.004444,0.004444
AGAP001097,2R,18496,19579,+,NaN,Elongation of very long chain fatty acids prot...,0.000000,0.044118,0.000000,0.004444
AGAP001098,2R,19797,21866,+,NaN,DAMOX [Source:VB Community Annotation],0.000000,0.044118,0.000000,0.004444
AGAP001099,2R,21882,31394,-,NaN,Polybromo-1 [Source:VB Community Annotation],0.000000,0.044118,0.004444,0.004444
...,...,...,...,...,...,...,...,...,...,...
AGAP004671,2R,61176240,61192895,+,NaN,nuclear pore complex protein Nup160 [Source:VB...,0.014706,0.029412,0.004444,0.000000
AGAP004672,2R,61230781,61254533,+,NaN,NaN,0.000000,0.044118,0.008889,0.004444
AGAP004674,2R,61303692,61306248,-,NaN,Phenoloxidase inhibitor protein [Source:VB Com...,0.000000,0.073529,0.000000,0.080000


### CNV coverage calls

Coverage calls can be accessed via the `cnv_coverage_calls()` method, which returns an xarray dataset. Note that an analysis parameter must be given, which can be either "arab" for *An. arabiensis* samples or "gamb_colu" for non-*An. arabiensis*. E.g.:

In [44]:
ds_cnv = ag3.cnv_coverage_calls(contig='2R', analysis='arab', sample_set='AG1000G-TZ')
ds_cnv

<xarray.Dataset>
Dimensions:              (samples: 221, variants: 9459)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(9459,), meta=np.ndarray>
    variant_end          (variants) int32 dask.array<chunksize=(9459,), meta=np.ndarray>
    variant_contig       (variants) uint8 dask.array<chunksize=(9459,), meta=np.ndarray>
    variant_id           (variants) object dask.array<chunksize=(9459,), meta=np.ndarray>
    sample_id            (samples) object dask.array<chunksize=(221,), meta=np.ndarray>
Dimensions without coordinates: samples, variants
Data variables:
    variant_CIPOS        (variants) int32 dask.array<chunksize=(9459,), meta=np.ndarray>
    variant_CIEND        (variants) int32 dask.array<chunksize=(9459,), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(9459,), meta=np.ndarray>
    call_genotype        (variants, samples) int8 dask.array<chunksize=(9459, 64), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

Here "variants" are copy number variants, inferred from the HMM results. CNV start positions are given by the `variant_position` array and ends are given by `variant_end`.

In [45]:
pos = ds_cnv['variant_position'].values
pos

array([       1,        1,      601, ..., 61514401, 61515901, 61533301],
      dtype=int32)

In [46]:
end = ds_cnv['variant_end'].values
end

array([    1500,     2100,     2100, ..., 61517400, 61517700, 61535700],
      dtype=int32)

CNV genotypes are given by the `call_genotype` array, coded as 0 for absence and 1 for presence of the CNV allele. Rows are CNV alleles and columns are individual samples.

In [47]:
gt = ds_cnv['call_genotype'].values
gt

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

Note that not all samples will have been included in the coverage calls, as some are excluded due to high coverage variance. Use the `sample_id` array to access the samples included in a given dataset, e.g.:

In [48]:
samples = ds_cnv['sample_id'].values
samples[:5]

array(['BL0047-C', 'BL0048-C', 'BL0049-C', 'BL0050-C', 'BL0055-C'],
      dtype=object)

### CNV discordant read calls

Access the discordant read calls via the `cnv_discordant_read_calls()` method. E.g.:

In [49]:
ds_cnv_dr = ag3.cnv_discordant_read_calls(contig="2R", sample_sets="AG1000G-TZ")
ds_cnv_dr

<xarray.Dataset>
Dimensions:                        (samples: 300, variants: 40)
Coordinates:
    variant_position               (variants) int32 dask.array<chunksize=(40,), meta=np.ndarray>
    variant_end                    (variants) int32 dask.array<chunksize=(40,), meta=np.ndarray>
    variant_id                     (variants) object dask.array<chunksize=(40,), meta=np.ndarray>
    variant_contig                 (variants) uint8 dask.array<chunksize=(40,), meta=np.ndarray>
    sample_id                      (samples) object dask.array<chunksize=(300,), meta=np.ndarray>
Dimensions without coordinates: samples, variants
Data variables:
    variant_Region                 (variants) object dask.array<chunksize=(40,), meta=np.ndarray>
    variant_StartBreakpointMethod  (variants) int32 dask.array<chunksize=(40,), meta=np.ndarray>
    variant_EndBreakpointMethod    (variants) int32 dask.array<chunksize=(40,), meta=np.ndarray>
    call_genotype                  (variants, samples) int8 dask.array<chunksize=(40, 64), meta=np.ndarray>
    sample_coverage_variance       (samples) float32 dask.array<chunksize=(300,), meta=np.ndarray>
    sample_is_high_variance        (samples) bool dask.array<chunksize=(300,), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

CNV start positions are given by the `variant_position` array and ends are given by `variant_end`. Note that some starts and/or ends are not known, and missing values are coded as -1.

The identifier of the CNV alleles is given by the `variant_id` array. The region where CNV alleles were analysed is given by the `variant_Region` array.

Note that for each region there is a special variant named "Dup0" which simply records whether any of the genes in a given cluster have increased coverage based on the modal HMM copy number state. I.e., if a sample has a "Dup0" variant but no other variant in a given region, then it has some form of copy number variation but we couldn't find any discordant reads to uniquely identify the CNV allele it carries.

CNV genotypes are given by the `call_genotype` array, coded as 0 for absence and 1 for presence of the CNV allele. Rows are CNV alleles and columns are individual samples.

In [50]:
vid = ds_cnv_dr['variant_id'].values
vid

array(['Ace1_Dup0', 'Cyp6aap_Dup0', 'Cyp6aap_Dup1', 'Cyp6aap_Dup23',
       'Ace1_Dup1', 'Ace1_Dup2', 'Ace1_Del1', 'Ace1_Del4', 'Ace1_Del3',
       'Ace1_Del2', 'Cyp6aap_Dup15', 'Cyp6aap_Dup13', 'Cyp6aap_Dup20',
       'Cyp6aap_Dup14', 'Cyp6aap_Dup12', 'Cyp6aap_Dup21', 'Cyp6aap_Dup19',
       'Cyp6aap_Dup8', 'Cyp6aap_Dup22', 'Cyp6aap_Dup17', 'Cyp6aap_Dup28',
       'Cyp6aap_Dup10', 'Cyp6aap_Dup7', 'Cyp6aap_Dup6', 'Cyp6aap_Dup4',
       'Cyp6aap_Dup30', 'Cyp6aap_Dup9', 'Cyp6aap_Dup3', 'Cyp6aap_Dup18',
       'Cyp6aap_Dup26', 'Cyp6aap_Dup1a', 'Cyp6aap_Dup1b', 'Cyp6aap_Dup25',
       'Cyp6aap_Dup5', 'Cyp6aap_Dup16', 'Cyp6aap_Dup24', 'Cyp6aap_Dup11',
       'Cyp6aap_Dup2', 'Cyp6aap_Dup29', 'Cyp6aap_Dup27'], dtype=object)

In [51]:
reg = ds_cnv_dr['variant_Region'].values
reg

array(['Ace1', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Ace1',
       'Ace1', 'Ace1', 'Ace1', 'Ace1', 'Ace1', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p', 'Cyp6aa/Cyp6p',
       'Cyp6aa/Cyp6p'], dtype=object)

In [52]:
pos = ds_cnv_dr['variant_position'].values
pos

array([      -1,       -1,       -1,       -1,  3436926,  3447950,
        3501850,  3512200,  3535850,  3539300, 28465673, 28472728,
       28473600, 28473874, 28474576, 28475128, 28475490, 28475996,
       28477220, 28477540, 28477710, 28477889, 28478057, 28478272,
       28478925, 28478987, 28479181, 28479407, 28479548, 28480166,
       28480189, 28480193, 28480335, 28480372, 28480547, 28480585,
       28487546, 28493547, 28494017, 28496700], dtype=int32)

In [53]:
end = ds_cnv_dr['variant_end'].values
end

array([      -1,       -1,       -1, 28497740,  3639836,  3518900,
        3599050,  3616290,  3619000,  3573750, 28555300, 28522671,
       28795255, 28563596, 28520016, 28483473, 28556726, 28485005,
       28484338, 28485380, 28496953, 28491215, 28486036, 28484157,
       28483069, 28485033, 28491431, 28483372, 28494597, 28483253,
       28483475, 28483675, 28483384, 28484518, 28483236, 28483442,
       28518123, 28497279, 28496505, 28499200], dtype=int32)

In [54]:
gt = ds_cnv_dr['call_genotype'].values
gt

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [55]:
samples = ds_cnv_dr['sample_id'].values
samples[:5]

array(['BL0046-C', 'BL0047-C', 'BL0048-C', 'BL0049-C', 'BL0050-C'],
      dtype=object)

## Haplotypes

The Ag3 data resource also includes haplotype reference panels, which were obtained by [phasing](https://en.wikipedia.org/wiki/Haplotype_estimation) the SNP calls. Phasing involves resolving the genotypes within each individual into a pair of haplotypes providing information about the two different DNA sequences inherited, one from each parent. Haplotypes provide greater power for a range of population genetic analyses, such as genome-wide scans for signals of recent selection, or analysis of adaptive gene flow between populations.

To allow for a range of different analyses, three different haplotype reference panels were constructed, each using a different subset of samples and applying different site filters:

* **"gamb_colu_arab"** - This haplotype reference panel includes **all wild-caught samples**, and phases biallelic SNPs passing the "gamb_colu_arab" site filters.
* **"gamb_colu"** - This haplotype reference panel includes **wild-caught samples assigned as either *An. gambiae*, *An. coluzzii* or intermediate *An. gambiae/An. coluzzii*** via the AIM species calling method, and phases biallelic SNPs passing the "gamb_colu" site filters.
* **"arab"** - This haplotype reference panel includes **wild-caught samples assigned as *An. arabiensis*** via the AIM species calling method, and phases biallelic SNPs passing the "arab" site filters. 

Data can be accessed in the cloud via the ``haplotypes()`` method. E.g., access haplotypes from the "gamb_colu_arab" analysis for all available samples, for chromosome arm 2R. This method returns an [xarray Dataset](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset).

In [56]:
ds_haps = ag3.haplotypes(analysis="gamb_colu_arab", contig="2R")
ds_haps

<xarray.Dataset>
Dimensions:           (alleles: 2, ploidy: 2, samples: 2784, variants: 16514919)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(262144,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(16514919,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(81,), meta=np.ndarray>
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(262144, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(262144, 64, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

Here we have haplotype data for 2,784 samples at 16,514,919 SNPs.

The SNP positions and alleles can be accessed as follows.

In [57]:
# access haplotype SNP positions
pos = ds_haps["variant_position"].data  # dask array
pos

dask.array<array, shape=(16514919,), dtype=int32, chunksize=(262144,), chunktype=numpy.ndarray>

In [58]:
# load positions of first 10 haplotype SNPs
pos[:10].compute()  # numpy array

array([135, 168, 170, 274, 289, 294, 295, 298, 299, 301], dtype=int32)

In [59]:
# access haplotype SNP alleles
alleles = ds_haps["variant_allele"].data  # dask array
alleles

dask.array<concatenate, shape=(16514919, 2), dtype=|S1, chunksize=(262144, 1), chunktype=numpy.ndarray>

In [60]:
# load alleles of first 10 haplotype SNPs - note all are biallelic
alleles[:10].compute()  # numpy array

array([[b'A', b'T'],
       [b'A', b'G'],
       [b'A', b'T'],
       [b'G', b'A'],
       [b'A', b'T'],
       [b'T', b'A'],
       [b'T', b'C'],
       [b'C', b'T'],
       [b'T', b'C'],
       [b'G', b'T']], dtype='|S1')

The phased genotypes can be accessed as follows.

In [61]:
# access genotypes
gt = ds_haps["call_genotype"].data  # dask array
gt

,Array,Chunk
Bytes,91.96 GB,33.55 MB
Shape,"(16514919, 2784, 2)","(262144, 64, 2)"
Count,7335 Tasks,3654 Chunks
Type,int8,numpy.ndarray


In [62]:
# wrap as scikit-allel genotype array
gt = allel.GenotypeDaskArray(ds_haps["call_genotype"].data)
gt

<GenotypeDaskArray shape=(16514919, 2784, 2) dtype=int8>

In [63]:
# convert to scikit-allel haplotype array - useful for some analyses
ht = gt.to_haplotypes()
ht

<HaplotypeDaskArray shape=(16514919, 5568) dtype=int8>

Note that in the haplotype array above, each row is a SNP and each column is a haplotype. There were $2784$ samples in this analysis, and so we have $5568$ ($2\times2784$) haplotypes.

### Using sample metadata to subset haplotypes

For some analyses, you'll want to subset the haplotypes, e.g., by location and species. In order to perform subsetting, you need to obtain sample metadata, and align it with the haplotype data. This ensures that every row in the sample metadata dataframe corresponds to every column in the phased genotypes array. E.g.:

In [64]:
# load sample metadata
df_samples = ag3.sample_metadata()
df_samples.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,release,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii,species
0,AR0047-C,LUA047,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.945,0.001,gamb_colu,coluzzii,coluzzii
1,AR0049-C,LUA049,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.933,0.001,gamb_colu,coluzzii,coluzzii
2,AR0051-C,LUA051,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.937,0.002,gamb_colu,coluzzii,coluzzii
3,AR0061-C,LUA061,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.938,0.002,gamb_colu,coluzzii,coluzzii
4,AR0078-C,LUA078,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.926,0.001,gamb_colu,coluzzii,coluzzii


In [65]:
# load IDs of phased samples
samples_phased = ds_haps['sample_id'].values
samples_phased[:5]

array(['AR0001-C', 'AR0002-C', 'AR0004-C', 'AR0006-C', 'AR0007-C'],
      dtype=object)

In [66]:
# align sample metadata to haplotypes
df_samples_phased = df_samples.set_index("sample_id").loc[samples_phased].reset_index()
df_samples_phased.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,release,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii,species
0,AR0001-C,LUA001,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.945,0.001,gamb_colu,coluzzii,coluzzii
1,AR0002-C,LUA002,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,M,AG1000G-AO,v3,0.939,0.002,gamb_colu,coluzzii,coluzzii
2,AR0004-C,LUA004,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,M,AG1000G-AO,v3,0.936,0.002,gamb_colu,coluzzii,coluzzii
3,AR0006-C,LUA006,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,M,AG1000G-AO,v3,0.938,0.002,gamb_colu,coluzzii,coluzzii
4,AR0007-C,LUA007,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,v3,0.940,0.001,gamb_colu,coluzzii,coluzzii


In [67]:
# now define some cohort of interest and locate samples
cohort_query = "country == 'Burkina Faso' and species == 'coluzzii' and year == 2012"
loc_cohort_samples = df_samples_phased.query(cohort_query).index.values
loc_cohort_samples

array([ 83,  84,  85,  86,  87,  88,  89,  90,  92,  93,  94,  95,  96,
        99, 100, 101, 102, 103, 104, 105, 110, 111, 112, 113, 120, 125,
       126, 127, 128, 129, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 184, 189, 190, 192, 193, 194, 195,
       196, 199, 200, 201, 202, 203, 205, 206, 208, 209, 213, 216, 219,
       221, 222, 224, 225, 226, 227, 228, 234, 235, 236, 239, 240, 243,
       244, 253, 256, 259])

In [68]:
# subset haplotypes to cohort
ds_haps_cohort = ds_haps.isel(samples=loc_cohort_samples)
ds_haps_cohort

<xarray.Dataset>
Dimensions:           (alleles: 2, ploidy: 2, samples: 82, variants: 16514919)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(262144,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(16514919,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(82,), meta=np.ndarray>
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(262144, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(262144, 30, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [69]:
# now access subsetted haplotypes
gt_cohort = allel.GenotypeDaskArray(ds_haps_cohort['call_genotype'].data)
ht_cohort = gt_cohort.to_haplotypes()
ht_cohort

<HaplotypeDaskArray shape=(16514919, 164) dtype=int8>

Note there are $82$ samples in the cohort and thus $164$ ($2\times82$) haplotypes. 

## Example computation

Here's an example computation to count the number of segregating SNPs on chromosome arm 3R that also pass gamb_colu_arab site filters. This may take a minute or two, because it is scanning genotype calls at millions of SNPs in hundreds of samples.

In [70]:
# choose chromosome arm
contig = "3R"

# choose site filter mask
mask = "gamb_colu_arab"

# choose sample sets
sample_sets = ["AG1000G-BF-A", "AG1000G-BF-B"]

# access SNP calls
ds_snps = ag3.snp_calls(contig=contig, sample_sets=sample_sets)

# locate pass sites
loc_pass = ds_snps[f"variant_filter_pass_{mask}"].values

# perform an allele count over genotypes
gt = allel.GenotypeDaskArray(ds_snps["call_genotype"].data)
with ProgressBar():
    ac = gt.count_alleles(max_allele=3).compute()

# locate segregating sites
loc_seg = ac.is_segregating()

# count segregating and pass sites
n_pass_seg = np.count_nonzero(loc_pass & loc_seg)

n_pass_seg

[########################################] | 100% Completed |  2min 28.4s


11226723

For more in-depth worked examples of various analyses, see the [Ag3 analysis examples](examples) page.

## Running larger computations

Please note that free cloud computing services such as Google Colab and MyBinder provide only limited computing resources. Thus although these services are able to efficiently read Ag3 data stored on Google Cloud, you may find that you run out of memory, or computations take a long time running on a single core. If you would like any suggestions regarding how to set up more powerful compute resources in the cloud, please feel free to get in touch via the [malariagen/vector-data GitHub discussion board](https://github.com/malariagen/vector-data/discussions).

## Feedback and suggestions

If there are particular analyses you would like to run, or if you have other suggestions for useful documentation we could add to this site, we would love to know, please get in touch via the [malariagen/vector-data GitHub discussion board](https://github.com/malariagen/vector-data/discussions).